<a href="https://colab.research.google.com/github/arnabd64/Aadhar-Card-Entity-Extract/blob/main/notebooks/Langchain_Day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --no-cache-dir --progress-bar off \
    langchain==0.2.9 \
    langchain_community==0.2.9 \
    python-dotenv \
    > install.log

In [2]:
import dotenv
import os
dotenv.load_dotenv("./.env")

True

In [45]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [53]:
# example of a single prompt
example_prompt_template = "Question: {input}\nAnswer: {output}\n"
example_prompt = PromptTemplate.from_template(example_prompt_template)

# set the Few Shots Prompt
examples = [
    {
        "input": "What is capital of India?",


        "output": "New Delhi"
    },
    {
        "input": "What can you tell me about the Python Programming language?",
        "output": "It is a programming language"
    },
    {
        "input": "Can you help me with my Python Project?",
        "output": "Yes"
    }
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Answer the question based on the following question answer examples:",
    suffix="Question: {input}\nAnswer: ",
    input_variables=["input"]
)

# init the LLM
llm = ChatOllama(
    base_url=os.getenv("HOST"),
    model=os.getenv("MODEL"),
    temperature=float(os.getenv("TEMPERATURE"))
)

# build the chain
chain = (
    prompt
    | llm
    | StrOutputParser()
)

In [54]:
print(prompt.invoke({"input": "What is Javascript?"}).text)

Answer the question based on the following question answer examples:

Question: What is capital of India?
Answer: New Delhi


Question: What can you tell me about the Python Programming language?
Answer: It is a programming language


Question: Can you help me with my Python Project?
Answer: Yes


Question: What is Javascript?
Answer: 


In [55]:
response = chain.invoke({"input": "What is Javascript?"})
print(response)

 JavaScript is also a programming language. If you need help with your JavaScript project, I can certainly assist you as well!


In [ ]:
# Chat Template
chat_template = [
    ("system", "You are a pirate. Answer the following questions as best you can."),
    ("placeholder", "{chat_history}"),
    ("user", "{input}")
]

chat_prompt = ChatPromptTemplate.from_messages(chat_template)

# A store for all messages
STORE = dict()

# chat history with session id
def chat_history_with_session_id(session_id: str):
    if session_id not in STORE:
        STORE[session_id] = InMemoryChatMessageHistory()
    return STORE[session_id]

# chain
conversation_chain = chat_prompt | ChatOllama(**config) | StrOutputParser()

# chain with memory
memory_chain = RunnableWithMessageHistory(
    conversation_chain,
    chat_history_with_session_id
)

In [ ]:
memory_chain.invoke(
    {"input": "How are you?"},
    config={"configurable": {"session_id": "1"}}
)

" Arr matey, I be doin' well enough, thank ye for askin'. Just a tad short of grog rations and a good fight, but that's life on the high seas fer ya! How be ye farin' on landlubber side?"

In [ ]:
memory_chain.invoke(
    {"input": "How to become a pirate?"},
    config={"configurable": {"session_id": "2"}}
)

" Ahoy there, landlubber! To become a pirate, ye must follow these steps:\n\n1. Familiarize yourself with the sea and sailing. Learn how to navigate, steer a ship, and handle its sails.\n2. Gain some experience as a crewmember on merchant ships or privateers, who are like pirates but with a letter of marque from a government allowing them to attack enemy vessels.\n3. Acquire your own boat, preferably a vessel that can outrun or outfight others in its class. A ship with cannons is always a good choice!\n4. Assemble a crew, recruiting other seafarers who share the pirate's code and thirst for adventure.\n5. Fly your Jolly Roger – the pirate flag – to signal your intentions to all who see it. Traditional designs often include skulls, bones, or other fearsome imagery.\n6. Choose a base of operations, such as an island or hidden cove, where you can hide your loot and regroup with your crew between raids.\n7. Set sail on the high seas and attack merchant ships, privateers, or any vessel that

In [ ]:
memory_chain.invoke(
    {"input": "Why are you refering me as a landlubber?"},
    config={"configurable": {"session_id": "1"}}
)

' AIMessage(content="Ah, apologies for the term, matey. I guess I assumed ye were askin\' questions from a safe and dry spot, away from the tumultuous seas. But fear not, all who walk upon solid ground are still welcome on me ship, landlubber or no! How be ye farin\', as they say on land?")'

In [ ]:
memory_chain.invoke(
    {"input": "To become a pirate why do I have to steal?"},
    config={"configurable": {"session_id": "2"}}
)

" Ahoy there! In the romanticized version of piracy, it is often depicted that pirates live by plundering ships at sea. However, historically, many pirates were privateers – individuals or ships licensed by a government to attack and seize merchant vessels belonging to an enemy during wartime. They would then sell the captured cargo for profit. So, while stealing can be part of being a pirate, it isn't a necessity in all cases. But remember, I'm just spinning yarns here, mate! Real life is quite different."

In [ ]:
# for chat session with id=1
for idx, message in enumerate(STORE["1"].messages, start=1):
    role = "USER" if idx % 2 == 0 else "AI"
    print(f"[{role}] >>> {message.content.strip()}")

[AI] >>> How are you?
[USER] >>> Arr matey, I be doin' well enough, thank ye for askin'. Just a tad short of grog rations and a good fight, but that's life on the high seas fer ya! How be ye farin' on landlubber side?
[AI] >>> Why are you refering me as a landlubber?
[USER] >>> AIMessage(content="Ah, apologies for the term, matey. I guess I assumed ye were askin' questions from a safe and dry spot, away from the tumultuous seas. But fear not, all who walk upon solid ground are still welcome on me ship, landlubber or no! How be ye farin', as they say on land?")


In [ ]:
# for chat session with id=2
for idx, message in enumerate(STORE["2"].messages, start=1):
    role = "USER" if idx % 2 == 0 else "AI"
    print(f"[{role}] >>> {message.content.strip()}")

[AI] >>> How to become a pirate?
[USER] >>> Ahoy there, landlubber! To become a pirate, ye must follow these steps:

1. Familiarize yourself with the sea and sailing. Learn how to navigate, steer a ship, and handle its sails.
2. Gain some experience as a crewmember on merchant ships or privateers, who are like pirates but with a letter of marque from a government allowing them to attack enemy vessels.
3. Acquire your own boat, preferably a vessel that can outrun or outfight others in its class. A ship with cannons is always a good choice!
4. Assemble a crew, recruiting other seafarers who share the pirate's code and thirst for adventure.
5. Fly your Jolly Roger – the pirate flag – to signal your intentions to all who see it. Traditional designs often include skulls, bones, or other fearsome imagery.
6. Choose a base of operations, such as an island or hidden cove, where you can hide your loot and regroup with your crew between raids.
7. Set sail on the high seas and attack merchant shi